# Looking at a Big Picture

What is Our Problem
Our problem is to predict whether the passenger with given set of characteristics will survive the crisis or not. We have to give answer the problem in Yes or No.

Select the Performance Measure
Our performance measure will be 'Accuracy'.
Accuracy = No. of correct predictions * 100/ Total number of predictions

Check the Assumptions
I) We are assuming that passenger will either survive or will die in the crisis. That is, the output will be binary only.